## 8. Robustness and Performance

### 68 Make `pickle` Reliable with `copyreg`

In [1]:
import logging

In [2]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4

In [3]:
state = GameState()
state.level += 1  # Player beat a level
state.lives -= 1  # Player had to try again

print(state.__dict__)

{'level': 1, 'lives': 3}


In [4]:
import pickle

In [5]:
state_path = 'game_state.bin'
with open(state_path, 'wb') as f:
    pickle.dump(state, f)

In [6]:
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)

print(state_after.__dict__)

{'level': 1, 'lives': 3}


In [7]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0  # New field

In [8]:
state = GameState()
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)

print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 0}


In [9]:
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)

print(state_after.__dict__)

{'level': 1, 'lives': 3}


In [10]:
assert isinstance(state_after, GameState)

In [11]:
class GameState:
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points

In [12]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)

In [13]:
def unpickle_game_state(kwargs):
    return GameState(**kwargs)

In [14]:
import copyreg

copyreg.pickle(GameState, pickle_game_state)

In [15]:
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)

print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 1000}


In [16]:
class GameState:
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic  # New field

In [17]:
print('Before:', state.__dict__)
state_after = pickle.loads(serialized)
print('After: ', state_after.__dict__)

Before: {'level': 0, 'lives': 4, 'points': 1000}
After:  {'level': 0, 'lives': 4, 'points': 1000, 'magic': 5}


In [18]:
class GameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

In [19]:
try:
    pickle.loads(serialized)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-19-c5ccc301c321>", line 2, in <module>
    pickle.loads(serialized)
  File "<ipython-input-13-844b36f90418>", line 2, in unpickle_game_state
    return GameState(**kwargs)
TypeError: __init__() got an unexpected keyword argument 'lives'


In [20]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2
    return unpickle_game_state, (kwargs,)

In [21]:
def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1:
        del kwargs['lives']
    return GameState(**kwargs)

In [22]:
copyreg.pickle(GameState, pickle_game_state)
print('Before:', state.__dict__)
state_after = pickle.loads(serialized)

print('After: ', state_after.__dict__)

Before: {'level': 0, 'lives': 4, 'points': 1000}
After:  {'level': 0, 'points': 1000, 'magic': 5}


In [23]:
copyreg.dispatch_table.clear()

In [24]:
state = GameState()
serialized = pickle.dumps(state)

In [25]:
del GameState

class BetterGameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

In [26]:
try:
    pickle.loads(serialized)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-26-c5ccc301c321>", line 2, in <module>
    pickle.loads(serialized)
AttributeError: Can't get attribute 'GameState' on <module '__main__'>


In [27]:
print(serialized)

b'\x80\x04\x95A\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tGameState\x94\x93\x94)\x81\x94}\x94(\x8c\x05level\x94K\x00\x8c\x06points\x94K\x00\x8c\x05magic\x94K\x05ub.'


In [28]:
copyreg.pickle(BetterGameState, pickle_game_state)

In [29]:
state = BetterGameState()
serialized = pickle.dumps(state)
print(serialized)

b'\x80\x04\x95W\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x13unpickle_game_state\x94\x93\x94}\x94(\x8c\x05level\x94K\x00\x8c\x06points\x94K\x00\x8c\x05magic\x94K\x05\x8c\x07version\x94K\x02u\x85\x94R\x94.'


> - 신뢰할 수 있는 프로그램 사이에 객체를 직렬화하고 역직렬화할 때는 `pickle` 내장 모듈이 유용하다.
> - 시간이 지남에 따라 클래스가 바뀔(애트리뷰트의 추가나 삭제 등) 수 있으므로 이전에 피크한 객체를 역직렬화하면 문제가 생길 수 있다.
> - 직렬화한 객체의 하위 호환성을 보장하고자 `copyreg` 내장 모듈과 `pickle`을 함께 사용하라.